In [1]:
from pyspark.sql import SparkSession
from PyClasses.Processes.Transformation import DE_Transformation
# Run Transformation
spark = SparkSession.builder.appName("DE-prj").getOrCreate()

transformation = DE_Transformation(spark)
cleanned_sentences, words = transformation.start_transform_rawdata()

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/16 23:05:25 WARN Utils: Your hostname, LoopZi. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/16 23:05:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/16 23:05:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-jav

In [2]:
from PyClasses.Processes.Transformation import DE_Transformation
filePaths = DE_Transformation.wordCountMapReduce()
#output at dir = 'hdfs://localhost:9000/user/student/DE-prj/MR_WC_Result/part-00000'

rm: `hdfs://localhost:9000/user/student/DE-prj/MR_WC_Result': No such file or directory


['/home/hduser/hadoop3/bin/hadoop', 'jar', '/home/hduser/hadoop3/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar', '-input', 'hdfs://localhost:9000/user/student/DE-prj/Words/part-00000-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00001-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00002-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00003-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00004-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00005-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00006-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/student/DE-prj/Words/part-00007-e028381a-45e4-4485-816c-b0cd23b6097f-c000.txt,hdfs://localhost:9000/user/stude